In [1]:

!pip install laserembeddings==1.1.2
#!python -m laserembeddings download-models
import torch
from tqdm.auto import tqdm
import re
tqdm.pandas()
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from numpy import dot
from numpy.linalg import norm
from laserembeddings import Laser
import site
import sys
import os
import gdown
import shutil
pd.set_option('display.max_colwidth', None)
#os.system('python -m laserembeddings download-models')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
torch.__version__

'1.12.1+cu113'

In [3]:
torch.cuda.is_available()

True

In [4]:

loc = site.getsitepackages() 
data_path = loc[0]+'/laserembeddings/data'
print(site.getusersitepackages()) # String for user-specific package location)
loc[0]

/root/.local/lib/python3.7/site-packages


'/usr/local/lib/python3.7/dist-packages'

In [5]:
# assets folder
url = "https://drive.google.com/drive/folders/1Zw64MRFvQxxwDLYFTdNki7HwNQOM30gy?usp=sharing"

id = "1Zw64MRFvQxxwDLYFTdNki7HwNQOM30gy"

In [6]:
%%time

dir = './assets/'
if not os.path.exists(dir):
    os.mkdir(dir)

files = os.listdir(dir)
if(len(files)<7):
    print("downloading necessary files....")
    gdown.download_folder(id=id, quiet=True, use_cookies=False)    

    shutil.copy('/content/assets/93langs.fcodes', data_path)
    shutil.copy('/content/assets/93langs.fvocab', data_path)
    shutil.copy('/content/assets/bilstm.93langs.2018-12-26.pt', data_path)


downloading necessary files....
CPU times: user 919 ms, sys: 894 ms, total: 1.81 s
Wall time: 18.2 s


In [7]:

#https://stackoverflow.com/questions/21030391/how-to-normalize-a-numpy-array-to-a-unit-vector
def normalized(a, axis=-1, order=2):
    l2 = np.atleast_1d(np.linalg.norm(a, order, axis))
    l2[l2==0] = 1
    return a / np.expand_dims(l2, axis)


def pairwise_dot(v1, v2):
    return (normalized(v1) * normalized(v2)).sum(axis=1)

def pairwise_euclidean_dists(x, y):
    dists = -2 * np.matmul(x, y.T)
    dists +=  np.sum(x**2, axis=1)[:, np.newaxis]
    dists += np.sum(y**2, axis=1)
    return  np.sqrt(dists)


laser = Laser()



In [8]:
corpus_emb_quran = np.load('./assets/Holy_Quran_mlt_emb.npy')
corpus_emb_hadith = np.load('./assets/en_emb_bukhari_muslim.npy')

en_bn_quran_tafsir = pd.read_csv('./assets/en_bn_quran_tafsir.csv')

en_bn_bukhari_muslim = pd.read_csv('./assets/en_bn_bukhari_muslim.csv')

en_bn_bukhari_muslim.head(1)

,id,hadith_id,source,chapter_no,hadith_no,chapter,chain_indx,text_ar,text_en,narrators,text_bn
0,0,1,Sahih Bukhari,1,1,Revelation,"30418, 20005, 11062, 11213, 11042, 3","حدثنا الحميدي عبد الله بن الزبير، قال حدثنا سفيان، قال حدثنا يحيى بن سعيد الأنصاري، قال أخبرني محمد بن إبراهيم التيمي، أنه سمع علقمة بن وقاص الليثي، يقول سمعت عمر بن الخطاب رضى الله عنه على المنبر قال سمعت رسول الله صلى الله عليه وسلم يقول ‏""‏ إنما الأعمال بالنيات، وإنما لكل امرئ ما نوى، فمن كانت هجرته إلى دنيا يصيبها أو إلى امرأة ينكحها فهجرته إلى ما هاجر إليه ‏""‏‏.‏","Narrated 'Umar bin Al-Khattab: I heard Allah's Apostle saying, ""The reward of deeds depends upon the intentions and every person will get the reward according to what he has intended. So whoever emigrated for worldly benefits or for a woman to marry, his emigration was for what he emigrated for.""","'Abdullah bin al Zubair bin 'Isa ,Sufyan bin 'Uyaynah ,Yahya bin Sa'id al Ansari ,Muhammad bin Ibrahim bin al Harith ,'Alqama bin Waqqas ,'Umar ibn al Khattab","উমর বিন আল-খাততাবের কাহিনীঃ আমি আল্লার রসূলকে বলতে শুনেছি, ""ক্রিয়া-কর্মের প্রতিদান উদ্দেশ্যের উপর নির্ভর করে এবং প্রত্যেক ব্যক্তিকে তার উদ্দেশ্য অনুযায়ী প্রতিদান দেওয়া হবে। সুতরাং যে কেউ পার্থিব জীবনোপকরণ বা বিবাহের উদ্দেশ্যে হিজরত করে, তার হিজরত তার হিজরতের জন্যই হবে।"


In [9]:
def MLT_Sahih_Hadith_Search_Engine(query,size=10,language = 'en',metric = 'dot',query_embedding=None):

    if(metric == 'dot'):
        query_embedding = np.squeeze(np.asarray(query_embedding))
        linear_similarities = pairwise_dot(corpus_emb_hadith, query_embedding)

    else:
        linear_similarities = pairwise_euclidean_dists(corpus_emb_hadith, query_embedding)
        linear_similarities = np.squeeze(np.asarray(linear_similarities))
        linear_similarities = np.array(linear_similarities, dtype=np.float32)

    if(metric == 'dot'):
        Top_index_doc = linear_similarities.argsort()[:-(size+1):-1]
    else:
        Top_index_doc = linear_similarities.argsort()[:-(size+1):]
        Top_index_doc = Top_index_doc[:size]

    
    linear_similarities.sort()
    find = pd.DataFrame()
    for i,index in enumerate(Top_index_doc):
        find.loc[i,'source'] = str(en_bn_bukhari_muslim['source'][index])
        find.loc[i,'chapter_no'] = str(en_bn_bukhari_muslim['chapter_no'][index])
        find.loc[i,'hadith_no'] = str(en_bn_bukhari_muslim['hadith_no'][index])
        find.loc[i,'chapter'] = str(en_bn_bukhari_muslim['chapter'][index])
        find.loc[i,'text_ar'] = str(en_bn_bukhari_muslim['text_ar'][index])
        find.loc[i,'text_en'] = str(en_bn_bukhari_muslim['text_en'][index])
        find.loc[i,'text_bn'] = str(en_bn_bukhari_muslim['text_bn'][index])
        find.loc[i,'narrators'] = str(en_bn_bukhari_muslim['narrators'][index])
        

    for j,simScore in enumerate(linear_similarities[:-(size+1):-1]):
        find.loc[j,'Score'] = simScore

    return find

In [10]:
en_bn_quran_tafsir.head(1)

,Name,Surah,Ayat,Verse,Tafseer,ar_text,আল_বায়ান,tafsir_bayan
0,The Opening,1,1,"In the name of Allah, the Beneficent, the Merciful.",In the Name of God the Compassionate the Merciful,بِسۡمِ اللّٰهِ الرَّحۡمٰنِ الرَّحِیۡمِ ﴿۱﴾\nبسم الله الرحمن الرحیم ﴿۱﴾,পরম করুণাময় অতি দয়ালু আল্লাহর নামে।,"(১) অনন্ত করুণাময় পরম দয়ালু আল্লাহর নামে (আরম্ভ করছি)।‘বিসমিল্লাহ’র পূর্বে ‘আক্বরাউ’ ‘আবদাউ’ অথবা ‘আতলু’ ফে’ল (ক্রিয়া) উহ্য আছে। অর্থাৎ, আল্লাহর নাম নিয়ে পড়ছি অথবা শুরু করছি কিংবা তেলাঅত আরম্ভ করছি। প্রত্যেক গুরুত্বপূর্ণ কাজ আরম্ভ করার পূর্বে ‘বিসমিল্লাহ’ পড়ার প্রতি তাকীদ করা হয়েছে। সুতরাং নির্দেশ করা হয়েছে যে, খাওয়া, যবেহ করা, ওযু করা এবং সহবাস করার পূর্বে ‘বিসমিল্লাহ’ পড়। অবশ্য ক্বুরআনে করীম তেলাঅত করার সময় ‘বিসমিল্লাহির রাহমানির রাহীম’ পড়ার পূর্বে ‘আউযু বিল্লাহি মিনাশ্শায়ত্বানির রাজীম’ পড়াও অত্যাবশ্যক। মহান আল্লাহ বলেছেন, ‘‘অতএব যখন তুমি ক্বুরআন পাঠ করবে, তখন বিতাড়িত শয়তান থেকে আল্লাহর আশ্রয় প্রার্থনা কর।’’ (সূরা নাহল ৯৮ আয়াত)।তাফসীরে আহসানুল বায়ান"


In [11]:
def Multilingual_Quran_Hadith_Search_Engine(query,size=10,language = 'en',metric = 'dot'):

    query_embedding = laser.embed_sentences(query, lang=language)

    mlt_hadiths = MLT_Sahih_Hadith_Search_Engine(query,size=size,language = 'en',metric = metric,query_embedding=query_embedding)
    
    if(metric == 'dot'):
        query_embedding = np.squeeze(np.asarray(query_embedding))
        linear_similarities = pairwise_dot(corpus_emb_quran, query_embedding)
   
    else:
        linear_similarities = pairwise_euclidean_dists(corpus_emb_quran, query_embedding)
        linear_similarities = np.squeeze(np.asarray(linear_similarities))
        linear_similarities = np.array(linear_similarities, dtype=np.float32)

    if(metric == 'dot'):
        Top_index_doc = linear_similarities.argsort()[:-(size+1):-1]
    else:
        Top_index_doc = linear_similarities.argsort()[:-(size+1):]
        Top_index_doc = Top_index_doc[:size]

  
    linear_similarities.sort()
    find = pd.DataFrame()
    for i,index in enumerate(Top_index_doc):
        find.loc[i,'Name'] = str(en_bn_quran_tafsir['Name'][index])
        find.loc[i,'Surah'] = str(en_bn_quran_tafsir['Surah'][index])
        find.loc[i,'Ayat'] = str(en_bn_quran_tafsir['Ayat'][index])
        find.loc[i,'Verse'] = str(en_bn_quran_tafsir['Verse'][index]) 
        find.loc[i,'Tafseer'] = str(en_bn_quran_tafsir['Tafseer'][index]) 
        find.loc[i,'ar_text'] = str(en_bn_quran_tafsir['ar_text'][index])
        
        find.loc[i,'আল_বায়ান'] = str(en_bn_quran_tafsir['আল_বায়ান'][index])
        find.loc[i,'tafsir_bayan'] = str(en_bn_quran_tafsir['tafsir_bayan'][index])

    for j,simScore in enumerate(linear_similarities[:-(size+1):-1]):
        find.loc[j,'Score'] = simScore
  
    return find,mlt_hadiths

In [12]:
%%time
result,mlt_hadiths = Multilingual_Quran_Hadith_Search_Engine("fire",size=5,language = 'en',metric = 'dot')

CPU times: user 439 ms, sys: 61.4 ms, total: 500 ms
Wall time: 516 ms


In [13]:
result

,Name,Surah,Ayat,Verse,Tafseer,ar_text,আল_বায়ান,tafsir_bayan,Score
0,The Calamity,101,11,Raging Fire.,It is A scorching fire of extremely hot temperature the hā’ of hiya is for consonantal quiescence and is retained when reciting without a subsequent pause or with; some omit it when reciting without a pause.,نَارٌ حَامِیَۃٌ ﴿۱۱﴾\nنار حامیۃ ﴿۱۱﴾,প্রজ্জ্বলিত অগ্নি।,"১১। তা অতি উত্তপ্ত অগ্নি।[1][1] যেমন হাদীস শরীফে আছে যে, দুনিয়াতে মানুষ যে আগুন ব্যবহার করে থাকে, তা জাহান্নামের ৭০ ভাগের এক ভাগ। জাহান্নামের আগুন দুনিয়ার আগুন হতে উষ্ণতার দিক দিয়ে ৬৯ গুণ বেশী। (সহীহ বুখারী মখলুক সৃষ্টি অধ্যায়, জাহান্নামের বিবরণ পরিচ্ছেদ, মুসলিম জান্নাতের বিবরণ অধ্যায়, জাহান্নামের আগুনের উষ্ণতা বিবরণ পরিচ্ছেদ)এক অন্য হাদীসে বর্ণিত হয়েছে যে, জাহান্নামের আগুন আল্লাহর নিকট অভিযোগ করে বলল যে, ‘আমার এক অংশ অপর অংশকে খেয়ে ফেলছে।’ তখন আল্লাহ জাহান্নামকে দুটি নিঃশ্বাস নিতে আদেশ করলেন। প্রথম নিঃশ্বাস হল গরমকালে। আর দ্বিতীয় নিঃশ্বাস হল শীতকালে। সুতরাং শীতকালে যে প্রচন্ড শীত অনুভব হয়, তা জাহান্নামের ঠান্ডা নিঃশ্বাসের কারণে। আর গ্রীষ্মকালে যে প্রচন্ড গরম পড়ে, তা জাহান্নামের গরম নিঃশ্বাসের ফলে। (বুখারী, উল্লিখিত বাবে)অন্য এক হাদীসে বর্ণিত হয়েছে, মহানবী (সাঃ) বলেছেন যে, ‘‘গরম যখন প্রচন্ড হয়, তখন নামায ঠান্ডা করে পড়। কেননা, গরমের প্রচন্ডতা জাহান্নামের উত্তেজনা থেকে হয়।’’ (প্রমাণ পূর্বে উল্লিখিত, মুসলিম মাসজিদ অধ্যায়)তাফসীরে আহসানুল বায়ান",0.327300
1,The Event,56,94,And roasting at hell-fire.,and a roasting in Hell-fire.,وَّ تَصۡلِیَۃُ جَحِیۡمٍ ﴿۹۴﴾\nو تصلیۃ جحیم ﴿۹۴﴾,আর জ্বলন্ত আগুনে প্রজ্জ্বলনে।,(৯৪) এবং জাহান্নাম প্রবেশ।-তাফসীরে আহসানুল বায়ান,0.318196
2,Yâ Sîn,36,1,Ya Sin.,Yā sīn God knows best what He means by these letters.,یٰسٓ ۚ﴿۱﴾\nیس ۚ﴿۱﴾,ইয়া-সীন।,"(১) ইয়া-সীন,[1][1] অনেকে এর অর্থ, ‘হে ব্যক্তি! বা হে মানুষ!’ বলেছেন। আবার অনেকে এটিকে নবী (সাঃ)-এর নাম এবং অনেকে আল্লাহ তাআলার সুন্দরতম নামাবলীর একটি নাম রূপে উল্লেখ করেছেন। কিন্তু এই সকল মত কোন প্রমাণ ছাড়াই পেশ করা হয়েছে। কারণ এটিও সেই ‘হরূফে মুক্বাত্ত্বাআত’ (বিচ্ছিন্ন অক্ষরমালা), যার অর্থ আল্লাহ ছাড়া অন্য কেউ অবগত নয়।তাফসীরে আহসানুল বায়ান",0.307974
3,The Cloaked One,74,5,Pollution shun!,and shun all defilement this rijz was explained by the Prophet s to be the graven images; in other words persist in shunning them.,وَ الرُّجۡزَ فَاهۡجُرۡ ۪﴿ۙ۵﴾\nو الرجز فاهجر ﴿ۙ۵﴾,আর অপবিত্রতা বর্জন কর।,"(৫) অপবিত্রতা বর্জন কর। [1][1] অর্থাৎ, মূর্তিপূজা ছেড়ে দাও। এটা আসলে রসূল (সাঃ)-এর মাধ্যমে লোকদেরকে নির্দেশ দেওয়া হচ্ছে।তাফসীরে আহসানুল বায়ান",0.266461
4,The Ascending Stairways,70,16,Eager to roast;,ripping out the scalp shawā is the plural of shawāt the skin of the head;,نَزَّاعَۃً لِّلشَّوٰی ﴿ۚۖ۱۶﴾\nنزاعۃ للشوی ﴿ۚۖ۱۶﴾,যা মাথার চামড়া খসিয়ে নেবে।,"(১৬) যা দেহ হতে চামড়া খসিয়ে দেবে। [1][1] অর্থাৎ, গোশত এবং চামড়াকে জ্বালিয়ে ছাই করে দিবে এবং মানুষ কেবল অস্থির কঙ্কালসার হয়ে অবশিষ্ট থাকবে।তাফসীরে আহসানুল বায়ান",0.248549


In [14]:
mlt_hadiths

,source,chapter_no,hadith_no,chapter,text_ar,text_en,text_bn,narrators,Score
0,Sahih Bukhari,24,1466,Obligatory Charity Tax Zakat,"وقال حنظلة عن طاوس، ‏""‏ جنتان ‏""‏‏.‏ وقال الليث حدثني جعفر، عن ابن هرمز، سمعت أبا هريرة رضى الله عنه عن النبي صلى الله عليه وسلم ‏""‏ جنتان ‏""‏‏.‏",See previous hadith.,পূর্ববর্তী হাদীস দেখুন।,"Hanzala bin Abi Sufyan bin 'Abdur Rahman ,al Laith bin Sa'd ,Ja'far bin Rabi'ya bin Shrhbyl ,'Abdur Rahman bin Harmaz al A'araj ,Abu Hurairah",0.286530
1,Sahih Muslim,2,114,The Book of Faith,وحدثني محمد بن حاتم، وعبد الرحمن بن بشر، قالا حدثنا بهز، حدثنا شعبة، حدثنا محمد بن عثمان بن عبد الله بن موهب، وأبوه، عثمان أنهما سمعا موسى بن طلحة، يحدث عن أبي أيوب، عن النبي صلى الله عليه وسلم بمثل هذا الحديث ‏.‏,This hadith is transmitted by Muhammad b. Hatim on the authority of Abu Ayyub Ansari.,এই হাদীসটি মুহাম্মদ (সা.) এর মাধ্যমে প্রেরিত হয়েছে। আবু আইয়ুব আনসারির কর্তৃত্বে হাতিম।,"Muhammad bin Hatm bin Sulaiman ,'Abdur Rahman bin Bashr ,Shu'bah bin al Hajjaj ,Muhammad bin 'Uthman al Akhnasi ,Musa ibn Talha ,Abu Ayyub al Ansari",0.280741
2,Sahih Muslim,34,4870,The Book on Government,وحدثناه زهير بن حرب، ومحمد بن المثنى، قالا حدثنا يحيى، وهو القطان ح وحدثنا ابن نمير، حدثنا أبي كلاهما، عن عبيد الله، بهذا الإسناد مثله ‏.‏,This hadith has been transmitted on the authority of 'Ubaidullah.,এই হাদীস উবাইদুল্লাহর কর্তৃত্বে প্রেরণ করা হয়েছে।,"Zuhayr bin Harb ,Muhammad bin al Muthna al Zaman ,Yahya bin Sa'id bin Farroukh al Qatan ,Yahya bin Sa'id bin Farroukh al Qatan ,Muhammad bin 'Abdullah bin Numayr",0.280198
3,Sahih Muslim,14,2607,The Book of Fasting,وحدثنا عمرو الناقد، حدثنا يزيد بن هارون، أخبرنا همام، ح وحدثنا ابن المثنى، حدثنا سالم بن نوح، حدثنا عمر بن عامر، كلاهما عن قتادة، بهذا الإسناد ‏.‏,This hadith has been transmitted on the authority of Qatada too.,এই হাদীস কাদাদের কর্তৃত্বেও প্রেরিত হয়েছে।,"'Amr bin Muhammad bin Bukayr al Naqid ,Yazid bin Harun ,Hmam bin Yahya bin Dinar ,Salm bin Nwh bin Aby",0.266573
4,Sahih Bukhari,8,490,Prayers Salat,وأن عبد الله بن عمر حدثه أن النبي صلى الله عليه وسلم كان ينزل في المسيل الذي في أدنى مر الظهران، قبل المدينة حين يهبط من الصفراوات ينزل في بطن ذلك المسيل عن يسار الطريق، وأنت ذاهب إلى مكة، ليس بين منزل رسول الله صلى الله عليه وسلم وبين الطريق إلا رمية بحجر‏.‏,See translation for hadith 484 above,উপরের ৪৮৪ হাদীসের অনুবাদ দেখুন,Not Found,0.264308
